In [45]:
import os
import pandas as pd
from llama_cloud_services import LlamaExtract
from dotenv import load_dotenv
from typing import Optional
from pydantic import ValidationError

from pydantic import BaseModel, Field
from typing import Optional, Type

from BS_Schema import make_StatementOfFinancialPosition_model

import BS_Schema
from importlib import reload

BS_Schema = reload(BS_Schema)

In [47]:
## Import Class
SFP = make_StatementOfFinancialPosition_model(2024)
# 4.2) Inspect its name and fields
print(SFP.__name__)           # → "StatementOfFinancialPosition_2024"
print(list(SFP.__fields__.keys()))

StatementOfFinancialPosition_2024
['year', 'cash_and_short_term_investments_unrestricted_and_restricted', 'accumulated_depreciation', 'net_fixed_assets', 'long_term_investments_unrestricted_and_restricted', 'total_assets', 'short_term_debt', 'long_term_debt', 'pension_and_opeb_liability', 'total_liabilities', 'net_assets_without_donor_restrictions', 'expendable_net_assets_with_donor_restrictions', 'perpetual_net_assets_with_donor_restrictions', 'net_assets_with_donor_restrictions', 'total_net_assets', 'total_liabilities_and_net_assets', 'net_receivables', 'rou_assets_finance_lease', 'rou_assets_operating_lease', 'other_assets', 'current_portion_finance_lease', 'current_portion_long_term_debt', 'current_portion_operating_lease', 'accounts_payable', 'deferred_revenue', 'long_term_finance_lease', 'long_term_operating_lease', 'swap_obligation_fmv', 'pension_liability', 'opeb_liability', 'other_liabilities', 'net_assets', 'noncontrolling_interest']


In [49]:
# ee231c03c0cf6607315be41f48f2ca75503002282bb13f37ff28a6197360edcd

# llx-zKFsBQYVXG6DqlTVV2yi3xpkS8nePwYvgOeYoMzJ04RBpgYw

#llx-85vV2s7NGkSWQevqJR6pn6I7hMgksTzbd9I4Vk6qIAPwtmUs - organization key

In [51]:
PDF_ROOT = "scrapping/university_pdfs/"
OUTPUT_ROOT = "output_Balance_Sheet_DS"
os.makedirs(OUTPUT_ROOT, exist_ok=True)  
AGENT_ID = "bcb15a18-67ac-4772-9965-2654ecaff88c" #Different based on your LLamaCloud account
load_dotenv() #make sure the API key is in the .env file
Year = 2024
extractor = LlamaExtract(api_key='llx-jlyOr0ZQwOzn0BPYgzggmo6mFShTLTsxdrXGviOaBf6IDnHG',project_id = '8c10e62e-3810-4193-915d-d2d11105826d')# LlamaExtract(project_id = '8c10e62e-3810-4193-915d-d2d11105826d')

# #uncomment the below line if you are creating the agent for the first time
# agent = extractor.create_agent(name = "balance-sheet-parser-v1", data_schema=SFP)

agent = extractor.get_agent(id = AGENT_ID)

# uncomment the following lines if you updated the schema
agent.data_schema = SFP
agent.save()
agent = extractor.get_agent(id = AGENT_ID)

In [53]:
agent.data_schema

{'additionalProperties': False,
 'properties': {'year': {'description': 'Only extract from the full Statement of Financial Position (Balance Sheet); do not use condensed or net financial position tables, and do not perform any calculations—capture the number exactly as shown. The fiscal year for all line‐items: 2024.',
   'type': 'integer'},
  'cash_and_short_term_investments_unrestricted_and_restricted': {'anyOf': [{'type': 'integer'},
    {'type': 'null'}],
   'description': 'Only extract from the full Statement of Financial Position (Balance Sheet); do not use condensed or net financial position tables, and do not perform any calculations—capture the number exactly as shown. Cash & Short-Term Investments (both unrestricted & restricted) as of June 30, 2024.'},
  'accumulated_depreciation': {'anyOf': [{'type': 'integer'},
    {'type': 'null'}],
   'description': 'Only extract from the full Statement of Financial Position (Balance Sheet); do not use condensed or net financial position

In [57]:
def process_school(school_name, school_dir):
    combined   = {}
    first_keys = None

    for fname in sorted(os.listdir(school_dir)):
        if not fname.lower().endswith(".pdf"):
            continue
        path = os.path.join(school_dir, fname)
        print(f"Extracting data from {fname}")
        try:
            run  = agent.extract(path)
            data = run.data or {}
            if first_keys is None:
                first_keys = list(data.keys())
                combined  = {k: None for k in first_keys}
            for k, v in data.items():
                if v not in (None, "", []):
                    combined[k] = v
        except Exception as err:
            print(f"Skipped {fname}: {err}")

    if first_keys:
        df = pd.DataFrame.from_dict(combined, orient="index", columns=["2024-25"])
        df.index.name = "Metric"
        outfile = os.path.join(OUTPUT_ROOT, f"{school_name}.xlsx")
        df.to_excel(outfile)
        print(f"Saved output to {outfile}")
    else:
        print(f"No PDF data found for {school_name}")

In [59]:
# os.listdir('/Users/danyalsoomro/Desktop/Python_Projects/Industry_projects/Franklin_Templton_Industry_Project/scrapping/university_pdfs')

In [65]:
# # Loop over schools
# for school in sorted(os.listdir(PDF_ROOT))[0:4]:
#     school_dir = os.path.join(PDF_ROOT, school)
#     if not os.path.isdir(school_dir):
#         continue
#     print(f"Processing school: {school}")
#     process_school(school, school_dir)

# for school in sorted(os.listdir(PDF_ROOT)):
#     school_dir = os.path.join(PDF_ROOT, school)
#     if not os.path.isdir(school_dir):
#         continue

#     # 1) Gather all PDFs in the directory
#     pdf_files = [
#         f for f in sorted(os.listdir(school_dir))
#         if f.lower().endswith(".pdf")
#     ]

#     # 2) Decide which PDFs to read:
#     if len(pdf_files) == 1:
#         to_read = pdf_files
#     else:
#         financial_only = [f for f in pdf_files if "financial" in f.lower()]
#         to_read = financial_only if financial_only else pdf_files

#     combined   = {}
#     first_keys = None

#     for fname in to_read:
#         path = os.path.join(school_dir, fname)
#         print(f"Extracting data from {school}/{fname}")
#         try:
#             run  = agent.extract(path)
#             data = run.data or {}
#             if first_keys is None:
#                 first_keys = list(data.keys())
#                 combined  = {k: None for k in first_keys}
#             for k, v in data.items():
#                 if v not in (None, "", []):
#                     combined[k] = v
#         except Exception as err:
#             print(f"Skipped {fname}: {err}")

#     if first_keys:
#         df = pd.DataFrame.from_dict(combined, orient="index", columns=["2024-25"])
#         df.index.name = "Metric"
#         sheet_name = school[:31]
#         df.to_excel(writer, sheet_name=sheet_name)
#     else:
#         print(f"No data for {school}.")


# print("Extraction complete.")

import os
import pandas as pd

results = {}

for school in sorted(os.listdir(PDF_ROOT)):
    school_dir = os.path.join(PDF_ROOT, school)
    if not os.path.isdir(school_dir):
        continue

    # 1) Gather all PDFs in the directory
    pdf_files = [
        f for f in sorted(os.listdir(school_dir))
        if f.lower().endswith(".pdf")
    ]

    # 2) Decide which PDFs to read:
    if len(pdf_files) == 1:
        to_read = pdf_files
    else:
        financial_only = [f for f in pdf_files if "financial" in f.lower()]
        to_read = financial_only if financial_only else pdf_files

    combined   = {}
    first_keys = None

    for fname in to_read:
        path = os.path.join(school_dir, fname)
        print(f"Extracting data from {school}/{fname}")
        try:
            run  = agent.extract(path)
            data = run.data or {}
            if first_keys is None:
                first_keys = list(data.keys())
                combined  = {k: None for k in first_keys}
            for k, v in data.items():
                if v not in (None, "", []):
                    combined[k] = v
        except Exception as err:
            print(f"Skipped {fname}: {err}")

    if first_keys:
        # store this school's combined dict in results
        results[school] = combined
    else:
        print(f"No data for {school}.")

# At this point `results` is a dict:
#   { "SchoolA": { metric1: val, metric2: val, … },
#     "SchoolB": { ... }, … }

# Build one DataFrame: rows=schools, columns=metrics
df_all = pd.DataFrame.from_dict(results, orient="index")

# Optionally rename the index
# df_all.index.name = "University"

# # Now `df_all` has shape (#schools x #metrics)
# print(df_all)


Extracting data from ARIZONA_STATE_UNIVERSITY/FY_2024_Arizona_State_University_Annual_Comprehensive_Financial_Report_for_the_year_ended_06_30_2024__4.1_MB_.pdf


Extracting files: 100%|███████████████████████████| 1/1 [00:24<00:00, 24.51s/it]


Extracting data from BRADLEY_UNIVERSITY/Annual_Financial_Information_and_Operating_Data__Rule_15c2-12__for_FY24_for_the_year_ended_05_31_2024__227_KB_.pdf


Extracting files: 100%|███████████████████████████| 1/1 [00:10<00:00, 10.21s/it]


Extracting data from BRADLEY_UNIVERSITY/Audited_Financial_Statements_or_ACFR__Rule_15c2-12__for_FY24_for_the_year_ended_05_31_2024__541_KB_.pdf


...

Extracting files: 100%|███████████████████████████| 1/1 [00:07<00:00,  7.40s/it]


Extracting data from UNIVERSITY_OF_MINNESOTA/Annual_Financial_Information_and_Operating_Data_for_the_year_ended_06_30_2024_Document1__1.3_MB_.pdf


Extracting files: 100%|███████████████████████████| 1/1 [00:15<00:00, 15.79s/it]


Extracting data from UNIVERSITY_OF_MINNESOTA/Annual_Financial_Information_and_Operating_Data_for_the_year_ended_06_30_2024_Document2__315_KB_.pdf


Extracting files: 100%|███████████████████████████| 1/1 [00:14<00:00, 14.37s/it]


In [67]:
# OUTPUT_FILE = os.path.join(OUTPUT_ROOT, "all_schools.xlsx")

# writer = pd.ExcelWriter(OUTPUT_FILE, engine="openpyxl")

# for school in sorted(os.listdir(PDF_ROOT)):
#     school_dir = os.path.join(PDF_ROOT, school)
#     if not os.path.isdir(school_dir):
#         continue

#     combined   = {}
#     first_keys = None
#     for fname in sorted(os.listdir(school_dir)):
#         if not fname.lower().endswith(".pdf"):
#             continue
#         path = os.path.join(school_dir, fname)
#         print(f"Extracting data from {school}/{fname}")
#         try:
#             run  = agent.extract(path)
#             data = run.data or {}
#             if first_keys is None:
#                 first_keys = list(data.keys())
#                 combined  = {k: None for k in first_keys}
#             for k, v in data.items():
#                 if v not in (None, "", []):
#                     combined[k] = v
#         except Exception as err:
#             print(f"Skipped {fname}: {err}")

#     if first_keys:
#         df = pd.DataFrame.from_dict(combined, orient="index", columns=["2024-25"])
#         df.index.name = "Metric"
#         sheet_name = school[:31]
#         df.to_excel(writer, sheet_name=sheet_name)
#     else:
#         print(f"No data for {school}.")

# writer.close()
# print(f"All schools written to {OUTPUT_FILE}")

In [79]:
df_all.to_excel(OUTPUT_FILE)

In [83]:

from typing import List

def add_plug_accounts(df: pd.DataFrame) -> pd.DataFrame:
    """
    Given a DataFrame where each row is a university and columns include:
      - 'total_assets', 'other_assets', [various asset components…]
      - 'total_liabilities', 'other_liabilities', [various liability components…]
    this will compute two new columns:
    
      * other_assets_plug      = total_assets - sum(all other asset components)
      * other_liabilities_plug  = total_liabilities - sum(all other liability components)
    
    so that:
      sum(assets components) + other_assets_plug == total_assets
      sum(liability components) + other_liabilities_plug == total_liabilities
    
    Returns the DataFrame with these two new columns.
    """
    # define the “known” component column names
    asset_components = [
        "cash_and_short_term_investments_unrestricted_and_restricted",
        "net_receivables",
        "net_fixed_assets",
        "long_term_investments_unrestricted_and_restricted",
        "rou_assets_finance_lease",
        "rou_assets_operating_lease"
    ]
    liability_components = [
        "short_term_debt",
        "current_portion_finance_lease",
        "current_portion_long_term_debt",
        "current_portion_operating_lease",
        "accounts_payable",
        "deferred_revenue",
        "long_term_debt",
        "long_term_finance_lease",
        "long_term_operating_lease",
        "swap_obligation_fmv",
        "pension_and_opeb_liability",
        "pension_liability",
        "opeb_liability"
    ]
    
    # pick only those actually in the DataFrame
    assets_cols = [c for c in asset_components if c in df.columns]
    liabs_cols  = [c for c in liability_components if c in df.columns]
    
    # compute the sum of components (skip NaNs)
    df["sum_asset_components"]     = df[assets_cols].sum(axis=1, skipna=True)
    df["sum_liability_components"] = df[liabs_cols].sum(axis=1, skipna=True)
    
    # now compute the plugs
    df["other_assets_plug"]     = df["total_assets"] - df["sum_asset_components"]
    df["other_liabilities_plug"] = df["total_liabilities"] - df["sum_liability_components"]
    
    # (optional) drop the intermediate sum columns if you like
    df.drop(columns=["sum_asset_components", "sum_liability_components"], inplace=True)
    
    return df


In [81]:
df_all

,year,cash_and_short_term_investments_unrestricted_and_restricted,accumulated_depreciation,net_fixed_assets,long_term_investments_unrestricted_and_restricted,total_assets,short_term_debt,long_term_debt,pension_and_opeb_liability,total_liabilities,...,accounts_payable,deferred_revenue,long_term_finance_lease,long_term_operating_lease,swap_obligation_fmv,pension_liability,opeb_liability,other_liabilities,net_assets,noncontrolling_interest
ARIZONA_STATE_UNIVERSITY,2024,6.994920e+05,2.426794e+06,4.087901e+06,1.948669e+06,6.952605e+06,NaN,2517149,7.844820e+05,4.478833e+06,...,139132.0,4.072660e+05,NaN,228002.0,2397.0,664611.0,1.193510e+05,49477.0,2.297576e+06,None
BRADLEY_UNIVERSITY,2024,1.751900e+04,2.663700e+05,3.257570e+05,3.624220e+05,7.169740e+05,231.0,136507,2.272600e+04,1.885740e+05,...,8213.0,2.358000e+03,252.0,1474.0,0.0,NaN,2.272600e+04,NaN,5.284000e+05,None
CALIFORNIA_STATE_UNIVERSITY,2024,5.793096e+06,1.132942e+07,1.265873e+07,2.537880e+06,2.226212e+07,273292.0,10074727,2.282082e+07,3.586452e+07,...,1693959.0,4.178380e+05,276484.0,276484.0,NaN,8856579.0,1.396434e+07,757153.0,-1.279277e+07,None
CORNELL_UNIVERSITY,2024,9.670800e+05,5.228567e+06,4.709867e+06,1.151051e+07,1.954862e+07,NaN,2497549,2.949360e+05,5.022353e+06,...,434514.0,7.057340e+05,54812.0,507596.0,86184.0,25766.0,2.691700e+05,235516.0,NaN,None
CULINARY_INSTITUTE_OF_AMERICA_THE,2024,5.418600e+07,2.267304e+08,2.532383e+08,1.778971e+08,5.545144e+08,NaN,109154318,NaN,1.674309e+08,...,3821198.0,2.687975e+07,511018.0,7414758.0,NaN,NaN,NaN,NaN,NaN,None
GANNON_UNIVERSITY,2024,3.113700e+04,1.260330e+05,1.862320e+05,9.479200e+04,3.247590e+05,NaN,88226,2.220000e+02,1.116260e+05,...,2252.0,5.288000e+03,NaN,1268.0,NaN,NaN,2.220000e+02,11957.0,2.131330e+05,None
LEWIS_UNIVERSITY,2024,8.907539e+06,1.700560e+08,1.613502e+08,1.223007e+08,3.256467e+08,NaN,73716872,NaN,1.034088e+08,...,4061814.0,1.723077e+06,365138.0,4788888.0,NaN,NaN,NaN,NaN,NaN,None
MICHIGAN_STATE_UNIVERSITY,2024,5.486030e+05,3.387000e+09,3.152000e+09,4.625000e+09,9.323000e+09,202306.0,2442000000,NaN,4.009000e+09,...,194857.0,1.305630e+05,49721.0,32490.0,9194.0,NaN,7.830000e+08,NaN,5.600000e+09,None
MOLLOY_COLLEGE,2024,4.372167e+07,1.100760e+05,1.224520e+05,5.796100e+04,2.559410e+05,NaN,77512094,NaN,1.259190e+05,...,11935066.0,3.787335e+06,165079.0,18606343.0,NaN,NaN,NaN,NaN,1.300220e+05,None
MOUNT_ST_MARY_S_UNIVERSITY_INC,2024,3.495924e+07,1.286170e+08,1.022145e+08,6.557365e+07,2.205235e+08,NaN,56139713,NaN,9.003690e+07,...,11262838.0,9.219582e+06,222167.0,5798305.0,NaN,NaN,NaN,9701231.0,NaN,None


In [85]:
df_allv1 = add_plug_accounts(df_all)
df_allv1

,year,cash_and_short_term_investments_unrestricted_and_restricted,accumulated_depreciation,net_fixed_assets,long_term_investments_unrestricted_and_restricted,total_assets,short_term_debt,long_term_debt,pension_and_opeb_liability,total_liabilities,...,long_term_finance_lease,long_term_operating_lease,swap_obligation_fmv,pension_liability,opeb_liability,other_liabilities,net_assets,noncontrolling_interest,other_assets_plug,other_liabilities_plug
ARIZONA_STATE_UNIVERSITY,2024,6.994920e+05,2.426794e+06,4.087901e+06,1.948669e+06,6.952605e+06,NaN,2517149,7.844820e+05,4.478833e+06,...,NaN,228002.0,2397.0,664611.0,1.193510e+05,49477.0,2.297576e+06,None,-4.803730e+05,-5.881060e+05
BRADLEY_UNIVERSITY,2024,1.751900e+04,2.663700e+05,3.257570e+05,3.624220e+05,7.169740e+05,231.0,136507,2.272600e+04,1.885740e+05,...,252.0,1474.0,0.0,NaN,2.272600e+04,NaN,5.284000e+05,None,3.390000e+03,-1.046800e+04
CALIFORNIA_STATE_UNIVERSITY,2024,5.793096e+06,1.132942e+07,1.265873e+07,2.537880e+06,2.226212e+07,273292.0,10074727,2.282082e+07,3.586452e+07,...,276484.0,276484.0,NaN,8856579.0,1.396434e+07,757153.0,-1.279277e+07,None,3.883220e+05,-2.327978e+07
CORNELL_UNIVERSITY,2024,9.670800e+05,5.228567e+06,4.709867e+06,1.151051e+07,1.954862e+07,NaN,2497549,2.949360e+05,5.022353e+06,...,54812.0,507596.0,86184.0,25766.0,2.691700e+05,235516.0,NaN,None,1.013241e+06,1.460920e+05
CULINARY_INSTITUTE_OF_AMERICA_THE,2024,5.418600e+07,2.267304e+08,2.532383e+08,1.778971e+08,5.545144e+08,NaN,109154318,NaN,1.674309e+08,...,511018.0,7414758.0,NaN,NaN,NaN,NaN,NaN,None,6.025793e+07,1.286212e+07
GANNON_UNIVERSITY,2024,3.113700e+04,1.260330e+05,1.862320e+05,9.479200e+04,3.247590e+05,NaN,88226,2.220000e+02,1.116260e+05,...,NaN,1268.0,NaN,NaN,2.220000e+02,11957.0,2.131330e+05,None,3.837000e+03,1.194200e+04
LEWIS_UNIVERSITY,2024,8.907539e+06,1.700560e+08,1.613502e+08,1.223007e+08,3.256467e+08,NaN,73716872,NaN,1.034088e+08,...,365138.0,4788888.0,NaN,NaN,NaN,NaN,NaN,None,4.688209e+06,1.875300e+07
MICHIGAN_STATE_UNIVERSITY,2024,5.486030e+05,3.387000e+09,3.152000e+09,4.625000e+09,9.323000e+09,202306.0,2442000000,NaN,4.009000e+09,...,49721.0,32490.0,9194.0,NaN,7.830000e+08,NaN,5.600000e+09,None,1.545114e+09,7.833246e+08
MOLLOY_COLLEGE,2024,4.372167e+07,1.100760e+05,1.224520e+05,5.796100e+04,2.559410e+05,NaN,77512094,NaN,1.259190e+05,...,165079.0,18606343.0,NaN,NaN,NaN,NaN,1.300220e+05,None,-6.374772e+07,-1.160444e+08
MOUNT_ST_MARY_S_UNIVERSITY_INC,2024,3.495924e+07,1.286170e+08,1.022145e+08,6.557365e+07,2.205235e+08,NaN,56139713,NaN,9.003690e+07,...,222167.0,5798305.0,NaN,NaN,NaN,9701231.0,NaN,None,7.833024e+06,5.999293e+06


In [92]:
df_allv1.to_excel(OUTPUT_FILE)